<a href="https://colab.research.google.com/github/Harshasingh5/Training/blob/master/Final_Tagging_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Cleaning data


In [0]:
import pandas as pd
# import spacy
import re
import string
#upload flashtext_me_12 file first
from flashtext_me_12 import KeywordProcessor
from nltk import WordNetLemmatizer
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
base_directory = "/content/drive/My Drive/long_search_attribute_identify/NOMBRENUMERUS/"
 ##to tokenise 
# nlp = spacy.load('en', disable=['parser', 'ner'])
numb = re.compile(r'\d[x|X|.|/|"|*|#|,|^|:|<|>|;|\s|\-|\d]*')
wnl = WordNetLemmatizer()


def regex_clean(key):
    key.translate(str.maketrans("","", string.punctuation)).strip()
    return key

def read_file(filename):
    kv_dict_file = base_directory+filename
    kv_dict = pd.read_csv(kv_dict_file, names=['FK_IM_SPEC_OPTIONS_DESC' ,'FK_IM_SPEC_MASTER_DESC'] )
    return kv_dict

def lemmatize(row_list):
    data = " ".join([wnl.lemmatize(word) for word in word_tokenize(row_list)])
    return data


def clean_searches(filename):
    searches_file = base_directory+filename
    searches_d = pd.read_csv(searches_file, names=['searches'], skiprows=1)
    print(searches_d)
    #fill NA with blank
    searches_d = searches_d.fillna("")
    
    ##lower case
    searches_d['searches'] = searches_d['searches'].str.lower().apply(lambda x: lemmatize(x)).str.replace(numb,'NOMBRENUMERUS').replace("\n", " ").replace("\t", " ").str.translate(str.maketrans("","", string.punctuation)).str.strip()
    ##drop duplicates
    searches = searches_d.drop_duplicates()
        
    return searches


def clean_ISQ_kv_dict(filename, key_name=None):
    kv_dict_F = pd.DataFrame()
    kv_dict_file = base_directory+filename
    import time
    start_time = time.time()
    print("--- %s seconds ---" % (time.time() - start_time))
    for kv_dict in pd.read_csv(kv_dict_file, names=['mcat_org_names','FK_IM_SPEC_MASTER_DESC','FK_IM_SPEC_OPTIONS_DESC'] , chunksize=3000000,  error_bad_lines=False, encoding="ISO-8859–1"):
        if key_name:
            kv_dict['FK_IM_SPEC_OPTIONS_DESC'] = key_name

        #fill NA with blank
        kv_dict = kv_dict.fillna("")
        print("--- %s seconds to read file and fillna---" % (time.time() - start_time))
        print("kv_dict:--------------", kv_dict)

        ##lower case
        kv_dict['FK_IM_SPEC_MASTER_DESC'] = kv_dict['FK_IM_SPEC_MASTER_DESC'].str.lower().apply(lambda x: lemmatize(x)).str.replace(numb,'NOMBRENUMERUS ').str.translate(str.maketrans("","", string.punctuation)).str.replace("\n", " ").str.replace("\t", " ").str.strip()
        kv_dict['FK_IM_SPEC_OPTIONS_DESC'] = kv_dict['FK_IM_SPEC_OPTIONS_DESC'].str.lower().apply(lambda x: lemmatize(x)).str.replace(numb,'NOMBRENUMERUS ').str.translate(str.maketrans("","", string.punctuation)).str.replace("\n", " ").str.replace("\t", " ").str.strip()
       
        print("--- %s seconds To number strip extra spaces---" % (time.time() - start_time))
        print("num kv_dict: --------------------------",kv_dict)

        ##drop duplicates
        kv_dict = kv_dict.drop_duplicates()   
        kv_dict_F = kv_dict_F.append(kv_dict)
        print("kv_dict_ dropped-----------------------------------------------------------------------------------------------------------------::::",kv_dict)

        with open('clean_data_split_10JULY.csv', 'a') as f:
            kv_dict.to_csv(f, header=False , columns=['FK_IM_SPEC_MASTER_DESC','FK_IM_SPEC_OPTIONS_DESC'])
        # print("kv_dict_F-----------------------------------------------------------------------------------------------------------------::::",kv_dict_F)
    return kv_dict  

def clean_searches(filename):
    searches_file = base_directory+filename
    searches_d = pd.read_csv(searches_file, names=['searches'], skiprows=1)
    print(searches_d)
    #fill NA with blank
    searches_d = searches_d.fillna("")
    
    ##lower case
    searches_d['searches'] = searches_d['searches'].str.lower().apply(lambda x: lemmatize(x)).str.replace(numb,'NOMBRENUMERUS ').replace("\n", " ").replace("\t", " ").str.translate(str.maketrans("","", string.punctuation)).str.strip()
    
    ##drop duplicates
    searches = searches_d.drop_duplicates()
    return searches


# searches_15k  clean_searches
isq_kv = clean_ISQ_kv_dict("brand_list.csv", "brand")
print("tada!!", isq_kv)
isq_kv.to_csv("brand_list_clean.csv", index=False , columns=['FK_IM_SPEC_MASTER_DESC','FK_IM_SPEC_OPTIONS_DESC'])

isq_kv = clean_searches("searches_15k.csv")
isq_kv.to_csv("searches_15k_clean.csv"



In [0]:
###Write the csv directly into google drive
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
# update_file = 'searches_15k_clean.csv'
update_file = 'brand_list_clean.csv'

auth.authenticate_user()

drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, mimetype='application/octet-stream',resumable=True)

    created = drive_service.files().create(body=file_metadata,media_body=media,fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created
save_file_to_drive(update_file, update_file)

#Tagging

**Time Taken by each cell:**

1.   Loading Data: 12.366s
2.   ISQ data to trie ds: 664.517s   RAM:7.3GB
3.   MCAT data to trie ds:  32.19s   RAM:0.22GB
4.   City data to trie ds:5.59s     RAM:0.03
5.   Tagging 15k searches: 3.64s    
6.   Writing in file:   RAM for 5 and 6: 0.02




In [0]:
import pandas as pd
# import spacy
import re
import string
from nltk import WordNetLemmatizer
from nltk import word_tokenize
import nltk

In [0]:
#if cleaned data exists
base_directory = "/content/drive/My Drive/long_search_attribute_identify/NOMBRENUMERUS/"
# base_directory = "/content/drive/My Drive/NOMBRENUMERUS/"
def read_files(filename):
    kv_dict_file = base_directory+filename
    kv_dict = pd.read_csv(kv_dict_file, names=['FK_IM_SPEC_OPTIONS_DESC' ,'FK_IM_SPEC_MASTER_DESC'])
    return kv_dict

def read_files_isqfreq(filename):
    kv_dict_file = base_directory+filename
    kv_dict = pd.read_csv(kv_dict_file, names=['FK_IM_SPEC_OPTIONS_DESC' ,'FK_IM_SPEC_MASTER_DESC', 'size'])
    return kv_dict

mcat_kv = read_files("clean_mcat_name_all.csv")
mcatalt_kv = read_files("mcat_alt_names_list_clean.csv")
brand_kv = read_files("brand_list_clean.csv")
isq_kv = read_files_isqfreq("grouped_all_isq.csv")
city_kv =  read_files("city.csv")
isq_kv = isq_kv.drop_duplicates()

searches_file = base_directory+"searches_15k_clean.csv"
# searches_file ="searches_15k_clean.csv"
searches = pd.read_csv(searches_file, names=['searches'])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [0]:
##For isqs
from flashtext_me_12 import KeywordProcessor
%time
exception_list = ["and","", "all", "NOMBRENUMERUS","by","supplier", "buyer", "any", "on", "to", "in", "for", "of","with","and","are", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
kp = KeywordProcessor()
for index,row in isq_kv.iterrows():
    if str(row['FK_IM_SPEC_OPTIONS_DESC']) not in exception_list:
        kp.add_keyword(str(row['FK_IM_SPEC_OPTIONS_DESC']), ( str(row['FK_IM_SPEC_MASTER_DESC']+"/"+str(row['size'])), str(row['FK_IM_SPEC_OPTIONS_DESC'])))
    # else:
        # print(row)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [0]:
##For mcat names
for index,row in mcatalt_kv.iterrows():
    if str(row['FK_IM_SPEC_OPTIONS_DESC']) not in exception_list:
        kp.add_keyword(str(row['FK_IM_SPEC_OPTIONS_DESC']), (str(row['FK_IM_SPEC_MASTER_DESC']), str(row['FK_IM_SPEC_OPTIONS_DESC'])))

In [0]:
##For mcat alt names
for index,row in mcat_kv.iterrows():
    if str(row['FK_IM_SPEC_OPTIONS_DESC']) not in exception_list:
        kp.add_keyword(str(row['FK_IM_SPEC_OPTIONS_DESC']), (str(row['FK_IM_SPEC_MASTER_DESC']), str(row['FK_IM_SPEC_OPTIONS_DESC'])))

In [0]:
###for city name
for index,row in city_kv.iterrows():
    if str(row['FK_IM_SPEC_OPTIONS_DESC']) not in exception_list:
        kp.add_keyword(str(row['FK_IM_SPEC_OPTIONS_DESC']), (str(row['FK_IM_SPEC_MASTER_DESC']), str(row['FK_IM_SPEC_OPTIONS_DESC'])))

In [0]:
###for brand name
for index,row in brand_kv.iterrows():
    if str(row['FK_IM_SPEC_OPTIONS_DESC']) not in exception_list:
        kp.add_keyword(str(row['FK_IM_SPEC_OPTIONS_DESC']), (str(row['FK_IM_SPEC_MASTER_DESC']), str(row['FK_IM_SPEC_OPTIONS_DESC'])))

In [0]:
##Tagging searches
searches_kv = [(kp.extract_keywords(str(row['searches'])), row['searches']) for index, row in searches.iterrows()]


In [0]:
#Write the tagged searches in csv
import pandas as pd

update_file = 'Tagged_searches_13Aug.csv'

for search in searches_kv:
    news = []  
    news.append(search[1])
    for sear in search[0]:
        testdf = pd.DataFrame(sear, columns=['isq','value'])
        isq = testdf['value'].unique()
        for each in isq:
            newdf = testdf[testdf['value'] == each]
            news.append(each)
            news.append(list(newdf['isq'].unique()))
    # print(news)
    with open(update_file, 'a') as f:
        pd.DataFrame([news]).to_csv(f, header=False,index=False)

In [0]:
###Write the csv directly into google drive
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
# update_file = 'searches_15k_clean.csv'
# update_file = 'WithFreq_tagged_searches.csv'

auth.authenticate_user()

drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
    file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

    media = MediaFileUpload(path, mimetype='application/octet-stream',resumable=True)

    created = drive_service.files().create(body=file_metadata,media_body=media,fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created
save_file_to_drive(update_file, update_file)

# Test/Rough


In [7]:
import re
numb = re.compile(r'\d[x|X|.|/|"|*|#|,|^|:|<|>|;|\s|\-|\d]*')
# "s hfj".str.replace(numb, "AOKAY")
re.sub(r'\d[x|X|.|/|"|*|#|,|^|:|<|>|;|\s|\-|\d]*' ,"AOKAY", "tv s")

'tv AOKAYs'

In [0]:
##For isqs
from flashtext_me_13Aug import KeywordProcessor
%time
exception_list = ["and","", "all", "NOMBRENUMERUS","by","supplier", "buyer", "any", "on", "to", "in", "for", "of","with","and","are", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
kn = KeywordProcessor()
for index,row in isq_kv.iterrows():
    if str(row['FK_IM_SPEC_OPTIONS_DESC']) not in exception_list:
        kn.add_keyword(str(row['FK_IM_SPEC_OPTIONS_DESC']), ( str(row['FK_IM_SPEC_MASTER_DESC']+"/"+str(row['size'])), str(row['FK_IM_SPEC_OPTIONS_DESC'])))
    # else:
        # print(row)

In [0]:
searches_kv = [(kn.extract_keywords(str(row['searches'])), row['searches']) for index, row in searches.iterrows()]


In [0]:

# kn.extract_keywords("currency note holy number of NOMBRENUMERUS")
kn.extract_keywords("tv apache rtr NOMBRENUMERUS spare part")

In [0]:
isq_kv.loc[isq_kv.FK_IM_SPEC_OPTIONS_DESC.str.contains("NOMBRENUMERUSin")]

,FK_IM_SPEC_OPTIONS_DESC,FK_IM_SPEC_MASTER_DESC,size


In [0]:
data = pd.read_csv('/content/WithFreq_tagged_searches.csv')

In [0]:
(kp.extract_keywords('NOMBRENUMERUS inch'))

In [0]:
import pandas as pd
# isq_kv = pd.read_csv('/content/drive/My Drive/long_search_attribute_identify/ISQ_kv_All.csv', error_bad_lines=False, encoding="ISO-8859–1", names=['FK_IM_SPEC_OPTIONS_DESC' ,'FK_IM_SPEC_MASTER_DESC'])
isq_kv = pd.read_csv('/content/drive/My Drive/long_search_attribute_identify/NOMBRENUMERUS/clean_ISQ_all_withDuplicates.csv')